<a href="https://colab.research.google.com/github/wojtekgradzinski/WojtekRepo/blob/main/FASHION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy 3.1

     |████████████████████████████████| 5.9 MB 5.0 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 623 kB 54.6 MB/s 
     |████████████████████████████████| 10.1 MB 32.9 MB/s 
     |████████████████████████████████| 456 kB 62.7 MB/s 
  Created wheel for 3.1: filename=3_1-1.0.0-py3-none-any.whl size=1340 sha256=3dbf7a73018955f2d340d7636215f726b853fb71df7c02d3024064450a9b55ac
  Stored in directory: /root/.cache/pip/wheels/a5/be/1f/0cfa3e1c197209b9c5c097f0e1d8712e240e12713ae61c7345
Successfully built 3.1
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uni

In [2]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.1 MB 9.9 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

nlp = spacy.load("en_core_web_lg")

In [16]:
#2nd model


with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.read()

doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

"""

STEP 1 - TRAIN DATA

"""

# Prepare training data

words = ["Gucci", "Chanel", "Schiaparelli", "Balenciaga",
         "Loewe", "Saint Laurent", "Atelier", "Versace",
         "Alberta Ferretti", "Lanvin", "Prada", "Jean Paul Gaultier",
         "Givenchy", "Louis Vuitton", "Burberry", "H&M","Ted Baker",
         "Armani Privé", "Emporio Armani", "Jayp", "Alexander McQueen", "Calvin Klein"]

Entities: [('Schiaparelli', 'FASHION'), ('Balenciaga', 'FASHION'), ('Gucci', 'FASHION'), ('Gucci', 'FASHION'), ('Gucci', 'FASHION'), ('Loewe', 'FASHION'), ('Gucci', 'FASHION'), ('Chanel', 'FASHION'), ('Saint Laurent', 'FASHION'), ('Atelier Versace', 'FASHION'), ('Armani Privé', 'FASHION'), ('Lanvin', 'FASHION'), ('Alberta Ferretti', 'FASHION'), ('Prada', 'FASHION'), ('Jean Paul Gaultier', 'FASHION'), ('Givenchy', 'FASHION'), ('Louis Vuitton', 'FASHION'), ('Burberry', 'FASHION'), ('Armani Privé', 'FASHION'), ('Armani Privé', 'FASHION'), ('Diodato', 'FASHION'), ('Emporio Armani', 'FASHION'), ('Emporio Armani', 'FASHION'), ('Armani Privé', 'FASHION'), ('Gessica Notaro', 'FASHION'), ('Jayp', 'FASHION'), ('Miu Miu', 'FASHION'), ('Alexander McQueen', 'FASHION'), ('Gucci', 'FASHION'), ('Chanel', 'FASHION'), ('Alexander McQueen', 'FASHION'), ('Calvin Klein', 'FASHION'), ('Chanel', 'FASHION'), ('Givenchy', 'FASHION'), ('Oscar De La Renta', 'FASHION'), ('Prada', 'FASHION'), ('Louis Vuitton', 'FA

In [17]:
train_data = []

with open("/content/drive/MyDrive/Colab Notebooks/7. Natural Language Processing/6. NER/fashion brands.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        print("######")
        print("sentence: ", sentence)
        print("######")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                print("word: ", word)
                print("----------------")
                print("start index:", start_index)
                print("end index:", end_index)
                pos = (start_index, end_index,'FASHION')
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        print('----------------')
        print("element:", element)

        # ("this is my sentence", {"entities": [0, 4, "PREP"]})
        # ("this is my sentence", {"entities": [6, 8, "VERB"]})


######
sentence:  Bella Hadid in Schiaparelli couture

######
word:  schiaparelli
----------------
start index: 15
end index: 27
----------------
element: ('bella hadid in schiaparelli couture', {'entities': [(15, 27, 'FASHION')]})
######
sentence:  

######
----------------
element: ('', {'entities': []})
######
sentence:  Meng Li in Balenciaga

######
word:  balenciaga
----------------
start index: 11
end index: 21
----------------
element: ('meng li in balenciaga', {'entities': [(11, 21, 'FASHION')]})
######
sentence:  

######
----------------
element: ('', {'entities': []})
######
sentence:  Jodie Turner-Smith in Gucci

######
word:  gucci
----------------
start index: 22
end index: 27
----------------
element: ('jodie turner-smith in gucci', {'entities': [(22, 27, 'FASHION')]})
######
sentence:  

######
----------------
element: ('', {'entities': []})
######
sentence:  Lou Doillon in Gucci

######
word:  gucci
----------------
start index: 15
end index: 20
----------------
eleme

In [18]:
"""

STEP 2 - UPDATE MODEL

"""

ner = nlp.get_pipe("ner")

for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        print("Iteration #", iteration)

        random.shuffle(train_data)
        losses = {}

        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], losses=losses, drop=0.1)
        print("Losses:", losses)

Iteration # 0
Losses: {'ner': 12.211947110204747}
Iteration # 1
Losses: {'ner': 2.0564684326615077}
Iteration # 2
Losses: {'ner': 0.10862482260811498}
Iteration # 3
Losses: {'ner': 10.512842909095607}
Iteration # 4
Losses: {'ner': 2.443315266180233}
Iteration # 5
Losses: {'ner': 1.8070172387495909}
Iteration # 6
Losses: {'ner': 4.8178691991130304e-06}
Iteration # 7
Losses: {'ner': 1.8194656119605325e-05}
Iteration # 8
Losses: {'ner': 5.565198629586918e-08}
Iteration # 9
Losses: {'ner': 2.188262904959357e-09}
Iteration # 10
Losses: {'ner': 3.3280263341726303e-10}
Iteration # 11
Losses: {'ner': 1.1239521986952813e-06}
Iteration # 12
Losses: {'ner': 1.2976930068504256e-08}
Iteration # 13
Losses: {'ner': 7.845005311445594e-09}
Iteration # 14
Losses: {'ner': 7.997723132965178e-09}
Iteration # 15
Losses: {'ner': 7.23279305548321e-08}
Iteration # 16
Losses: {'ner': 2.5000742659818503e-08}
Iteration # 17
Losses: {'ner': 8.867898448734897e-10}
Iteration # 18
Losses: {'ner': 4.916256305133582e-0

In [19]:
# Save the model

output_dir = Path("/ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")


Saved correctly!


In [22]:
"""

STEP 3 - TEST THE UPDATED MODEL

"""

print("Loading model...")
nlp_updated = spacy.load(output_dir)

# old sentence, old word


doc = nlp_updated("Cynthia Erivo attends the 92nd Academy Awards Designer: Versace  Year: 2020.")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# old sentence, new word
doc = nlp_updated("Janelle Monae attends the 92nd Academy Awards, Designer: Ralph Lauren, Year: 2000")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# new sentence, new word
doc = nlp_updated("I am a big fan of Tommy Hilfigher")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# new sentence, no word
doc = nlp_updated("Wojtek loves trading crypto!")
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

Loading model...
entities: [('Versace', 'FASHION')]
entities: []
entities: [('Tommy Hilfigher', 'FASHION')]
entities: []


In [13]:
print(s)

cynthia erivo attends the 92nd academy awards, designer: versace, year: 2020.
